In [1]:
import numpy as np
#import pytest
import tensorflow as tf

import hls4ml

2024-11-14 11:15:04.316730: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-14 11:15:04.351836: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-14 11:15:04.859109: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Keras implementation of a custom layer
class KReverse(tf.keras.layers.Layer):
    '''Keras implementation of a hypothetical custom layer'''

    def __init__(self):
        super().__init__()

    def call(self, inputs):
        return tf.reverse(inputs, axis=[-1])

    def get_config(self):
        # Breaks serialization and parsing in hls4ml if not defined
        return super().get_config()


# hls4ml layer implementation
class HReverse(hls4ml.model.layers.Layer):
    '''hls4ml implementation of a hypothetical custom layer'''

    def initialize(self):
        inp = self.get_input_variable()
        shape = inp.shape
        dims = inp.dim_names
        self.add_output_variable(shape, dims)

In [3]:
# Parser for converter
def parse_reverse_layer(keras_layer, input_names, input_shapes, data_reader):
    layer = {}
    layer['class_name'] = 'HReverse'
    layer['name'] = keras_layer['config']['name']
    layer['n_in'] = input_shapes[0][1]

    if input_names is not None:
        layer['inputs'] = input_names

    return layer, [shape for shape in input_shapes[0]]

In [4]:
rev_config_template = """struct config{index} : nnet::reverse_config {{
    static const unsigned n_in = {n_in};
}};\n"""

rev_function_template = 'nnet::reverse<{input_t}, {config}>({input}, {output});'
rev_include_list = ['nnet_utils/nnet_reverse.h']


class HReverseConfigTemplate(hls4ml.backends.template.LayerConfigTemplate):
    def __init__(self):
        super().__init__(HReverse)
        self.template = rev_config_template

    def format(self, node):
        params = self._default_config_params(node)
        return self.template.format(**params)


class HReverseFunctionTemplate(hls4ml.backends.template.FunctionCallTemplate):
    def __init__(self):
        super().__init__(HReverse, include_header=rev_include_list)
        self.template = rev_function_template

    def format(self, node):
        params = self._default_function_params(node)
        return self.template.format(**params)

In [5]:
# hls4ml optimizer to remove duplicate optimizer
class RemoveDuplicateReverse(hls4ml.model.optimizer.OptimizerPass):
    '''OptimizerPass to remove consecutive HReverse layers.'''

    def match(self, node):
        return isinstance(node, HReverse) and isinstance(node.get_input_node(), HReverse)

    def transform(self, model, node):
        first = node.get_input_node()
        second = node

        model.remove_node(first, rewire=True)
        model.remove_node(second, rewire=True)
        return True

In [6]:
# Register the converter for custom Keras layer
hls4ml.converters.register_keras_layer_handler('KReverse', parse_reverse_layer)

# Register the hls4ml's IR layer
hls4ml.model.layers.register_layer('HReverse', HReverse)

for backend_id in ['Vivado', 'Quartus']:
    # Register the optimization passes (if any)
    backend = hls4ml.backends.get_backend(backend_id)
    backend.register_pass('remove_duplicate_reverse', RemoveDuplicateReverse, flow=f'{backend_id.lower()}:optimize')

    # Register template passes for the given backend
    backend.register_template(HReverseConfigTemplate)
    backend.register_template(HReverseFunctionTemplate)

    # Register HLS implementation
    backend.register_source('/home/crchen/hls4ml-tutorial/nnet_reverse.h')

In [7]:
# Test if it works
kmodel = tf.keras.models.Sequential(
    [
        tf.keras.layers.Input(shape=(8,)),
        KReverse(),
        tf.keras.layers.ReLU(),
    ]
)

x = np.random.randint(-5, 5, (8,), dtype='int32')
kres = kmodel(x)

2024-11-14 11:16:03.217725: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [13]:
x = np.random.randint(-5, 5, (8,), dtype='int32')
print(x)

[ 2 -1 -3  2 -5  0  0 -2]


In [22]:




hmodel = hls4ml.converters.convert_from_keras_model(
    kmodel,
    output_dir=str(f'hls4mlprj_extensions_{backend_id}'),
    backend='Vitis',
    io_type='io_parallel',
    hls_config={'Model': {'Precision': 'ap_int<6>', 'ReuseFactor': 1}},
)

hmodel.compile()
hres = hmodel.predict(x.astype('float32'))

np.testing.assert_array_equal(kres, hres)

Interpreting Sequential
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 8]], output shape: [None, 8]
Layer name: k_reverse, layer type: HReverse, input shapes: [[None, 8]], output shape: [None, 8]
Layer name: re_lu, layer type: Activation, input shapes: [[None, 8]], output shape: [None, 8]
Creating HLS model
Writing HLS project
Done


AssertionError: 
Arrays are not equal

Mismatched elements: 7 / 8 (87.5%)
Max absolute difference: 7.
Max relative difference: 1.
 x: array([0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)
 y: array([3., 5., 5., 0., 7., 2., 4., 7.], dtype=float32)

In [ ]:
hres = hmodel.predict(x.astype('float32'))

In [ ]:
hres

In [31]:
hmodel._compile()

In [32]:
hres = hmodel.predict(x.astype('float32'))

In [33]:
hres

array([3., 5., 5., 0., 7., 2., 4., 7.], dtype=float32)

In [34]:
hmodel

In [35]:
import os

os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']

In [36]:
hmodel.build.__dict__

{}

In [37]:
report = hmodel.build(csim=False, synth=True, cosim=True , vsynth=True, export = True)


****** Vitis HLS - High-Level Synthesis from C, C++ and OpenCL v2023.2 (64-bit)
  **** SW Build 4023990 on Oct 11 2023
  **** IP Build 4028589 on Sat Oct 14 00:45:43 MDT 2023
  **** SharedData Build 4025554 on Tue Oct 10 17:18:54 MDT 2023
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.
    ** Copyright 2022-2023 Advanced Micro Devices, Inc. All Rights Reserved.

source /home/share/Xilinx/Vitis_HLS/2023.2/scripts/vitis_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/home/share/Xilinx/Vitis_HLS/2023.2/bin/unwrapped/lnx64.o/vitis_hls'
INFO: [HLS 200-10] For user 'crchen' on host 'Kaohsiung' (Linux_x86_64 version 5.15.0-100-generic) on Thu Nov 14 11:41:43 CST 2024
INFO: [HLS 200-10] On os Ubuntu 20.04.6 LTS
INFO: [HLS 200-10] In directory '/home/crchen/hls4ml-tutorial/hls4mlprj_extensions_Quartus'
INFO: [HLS 200-2053] The vitis_hls executable is being deprecated. Consider using vitis-run --mode hls --tcl
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-1510] Running: open